<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/Finetuning_Embeddings_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer,SentenceTransformerTrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss

In [3]:
# 1. Load a model to finetune
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

find all models at https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [4]:
# Calculate the similarity score between the two encoded sentences via the base model.

similarity = model.similarity(model.encode("What is the process of electrolysis?"),
                              model.encode("Electrolysis is a method of using a direct electric current to drive an otherwise non-spontaneous chemical reaction."))

print(similarity)

tensor([[0.7833]])


In [5]:
# 2. Load a dataset to finetune on and split into train and val sets
dataset = load_dataset("Mihaiii/qa-assistant")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/501k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5771 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1399 [00:00<?, ? examples/s]

In [6]:
train_dataset

Dataset({
    features: ['question', 'answer', 'score'],
    num_rows: 5771
})

In [7]:
# 3. Define a loss function
loss = CosineSimilarityLoss(model)

More information about the loss at https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosinesimilarityloss

In [8]:
args = SentenceTransformerTrainingArguments(
    output_dir="models/finetuned-all-mpnet-base-v2", # Directory to save the fine-tuned model
    num_train_epochs=5, # Number of training epochs
    per_device_train_batch_size=16, # Batch size per device during training
    per_device_eval_batch_size=16, # Batch size per device during evaluation
    warmup_ratio=0.1, # Fraction of training steps used for a linear learning rate warmup
    fp16=True, # Use mixed precision training (FP16) for faster computations and reduced memory usage
    # Evaluation strategy: how often to evaluate the model
    eval_strategy="steps",  # Evaluate after a specified number of steps
    eval_steps=100, # Perform evaluation every 100 steps
    save_strategy="steps", # Save model checkpoint strategy: save after a specified number of steps
    save_steps=100, # Save model checkpoint every 100 steps
    save_total_limit=2, # Limit the total number of saved checkpoints to 2
    logging_steps=1, # Log training metrics every 1 step
    report_to="none" # Disable reporting to online trackers (e.g., WandB, TensorBoard)
)


In [9]:
# 4. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)

trainer.train()

Step,Training Loss,Validation Loss
100,0.033300,0.038974
200,0.038500,0.036897
300,0.036500,0.036600
400,0.023200,0.034404
500,0.017300,0.034406
600,0.022600,0.032303
700,0.020400,0.031524
800,0.008100,0.031167
900,0.009400,0.030753
1000,0.015900,0.030011


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1805, training_loss=0.015971875737854628, metrics={'train_runtime': 467.0568, 'train_samples_per_second': 61.78, 'train_steps_per_second': 3.865, 'total_flos': 0.0, 'train_loss': 0.015971875737854628, 'epoch': 5.0})

In [10]:
# Calculate the similarity score between the two encoded sentences via the finetuned model.

similarity = model.similarity(model.encode("What is the process of electrolysis?"),
                              model.encode("Electrolysis is a method of using a direct electric current to drive an otherwise non-spontaneous chemical reaction."))

print(similarity)

tensor([[0.9534]])


As seen here, the model is now able to calculate similarity between these two sentences more accurately.